In [9]:
import numpy as np
import spacy
from collections import Counter, defaultdict
from sklearn.preprocessing import normalize
import settings
import json
from tags import *

In [10]:
data = json.load(open('data/coco_noun.tags'))

In [3]:
nlp = spacy.load(settings.SPACY_MODEL)
extractor = NounTags(nlp, alpha=1)

# Example

In [4]:
id_ = '384029'
caption_list = data['train2014'][id_]['captions']
caption_list

['A man preparing desserts in a kitchen covered in frosting.',
 'A chef is preparing and decorating many small pastries.',
 'A baker prepares various types of baked goods.',
 'a close up of a person grabbing a pastry in a container',
 'Close up of a hand touching various pastries.']

In [5]:
tags, rlocs = extractor.process(caption_list)
print("Tags: {}".format(tags))
print("rlocs: {}".format([round(r, 5) for r in rlocs]))

Tags: ['pastry', 'frosting', 'hand', 'good', 'man', 'type', 'close', 'container', 'kitchen', 'baker', 'chef', 'person', 'dessert']
rlocs: [0.2, 0.06667, 0.06667, 0.06667, 0.06667, 0.06667, 0.06667, 0.06667, 0.06667, 0.06667, 0.06667, 0.06667, 0.06667]


## Merged

In [6]:
captions = [nlp(c) for c in caption_list]

In [7]:
token_dict = defaultdict(set)
for sent in captions:
    for token in sent:
        if token.pos_ in ['NOUN', 'VERB']:
            token_dict[token.lemma_, token.pos_].add(token)
token_dict = dict(token_dict)

for k in list(token_dict):
    if len(token_dict[k]) == 1:
        del token_dict[k]
print('Aligned tokens: {}'.format(token_dict))

Aligned tokens: {('pastry', 'NOUN'): {pastry, pastries, pastries}, ('prepare', 'VERB'): {preparing, preparing, prepares}}


In [8]:
# build children token dict indexed by syntactic function
child_dict = defaultdict(set)
for k, tokens in token_dict.items():
    for token in tokens:
        for ctoken in token.children:
            if ctoken.pos_ in ['NOUN', 'VERB']:
                child_dict[k, ctoken.dep_].add(ctoken)
child_dict = dict(child_dict)

# keep only repeated children
for k in list(child_dict):
    if len(child_dict[k]) == 1:
        del child_dict[k]

print('Aligned children: {}'.format(child_dict))

Aligned children: {(('prepare', 'VERB'), 'nsubj'): {chef, baker}, (('prepare', 'VERB'), 'dobj'): {types, desserts}}


In [66]:
remove_idxs = set()
for tokens in child_dict.values():
    idxs = set(tags.index(t.lemma_) for t in tokens if t.lemma_ in tags)
    remove_idxs.update(sorted(idxs)[1:])

new_tags = [t for i, t in enumerate(tags) if i not in remove_idxs]
new_rlocs = [r for i, r in enumerate(rlocs) if i not in remove_idxs]

print('Final tags: {}'.format(' '.join(new_tags)))
print('Final rlocs: {}'.format(new_rlocs))

Final tags: pastry frosting good baker person container dessert kitchen man hand close
Final rlocs: [3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [50]:
c1 = captions[0]
c1

A man preparing desserts in a kitchen covered in frosting.

In [64]:
from nltk import Tree

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

for c in captions:
    for sent in c.sents:
        print()
        to_nltk_tree(sent.root).pretty_print()

          man                              
  _________|_________                       
 |   |           preparing                 
 |   |      _________|________              
 |   |     |                  in           
 |   |     |                  |             
 |   |     |               kitchen         
 |   |     |          ________|_______      
 |   |     |         |             covered 
 |   |     |         |                |     
 |   |     |         |                in   
 |   |     |         |                |     
 A   .  desserts     a             frosting

        preparing                           
  __________|___________________             
 |   |      |      |        decorating      
 |   |      |      |            |            
 |   |      |     chef       pastries       
 |   |      |      |     _______|________    
 is and     .      A   many            small

    prepares                    
  _____|_______________          
 |     |             types      
 |